In [ ]:
import os
import warnings
import pandas as pd
import numpy as np
import seaborn as sns

import sklearn as skl
import matplotlib.pyplot as plt
import sklearn.metrics as skmr
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from tabpfn import TabPFNClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.utils import check_random_state

from IPython.display import clear_output

In [ ]:
X_train = np.load("../TabPFN/mnist_train_X.npy")
y_train = np.load("../TabPFN/mnist_train_y.npy")
X_test = np.load("../TabPFN/mnist_test_X.npy")
y_test = np.load("../TabPFN/mnist_test_y.npy")
rng = check_random_state(42)
A = rng.normal(size=(28*28, 11))
test_samples = rng.choice(X_test.shape[0], 100)
XX_test = X_test[test_samples, :, :]
yy_test = y_test[test_samples]

In [ ]:
methods = [
    ("SVC", SVC()),
    ("LR", skl.linear_model.LogisticRegression()),
    #("kNN", KNeighborsClassifier()),
    ("XGBoost", xgb.XGBClassifier()),
    ("TabPFN", TabPFNClassifier(device='cpu')),
]

n_random = 10
n_samples_list = [30, 100, 300, 1000]
n_n_samples = len(n_samples_list)
accs = np.zeros((len(methods), n_n_samples, n_random))
f1s = np.zeros((len(methods), n_n_samples, n_random))
for nix, n_samples in enumerate(n_samples_list):
    for rix in range(n_random):
        _, cur_X_train, _, cur_y_train = train_test_split(
            X_train, y_train, test_size=n_samples / 60000, random_state=rix, stratify=y_train)
        rng = check_random_state(rix)
        for mix, (mname, method) in enumerate(methods):
            model = method
            model.fit(cur_X_train.reshape(-1, 28*28) @ A, cur_y_train)
            y_test_pred = model.predict(XX_test.reshape(-1, 28*28) @ A)
            accs[mix, nix, rix] = skmr.accuracy_score(yy_test, y_test_pred)
            f1s[mix, nix, rix] = skmr.f1_score(yy_test, y_test_pred, average="macro")
clear_output()

In [ ]:
fig, axes = plt.subplots(figsize=(5, 7), nrows=4, sharex=True);
for nix, n_samples in enumerate(n_samples_list):
    cur_accs = accs[:, nix, :]
    accs_df = pd.DataFrame(data=cur_accs, index=[mname for mname, _ in methods]).T
    sns.kdeplot(data=accs_df, clip=(0, 1), ax=axes[nix], legend=nix == 3)
    if nix == 3:
        axes[nix].set_xlabel("Accuracy");
    axes[nix].set_title(f"{n_samples} training samples");
    axes[nix].set_xticks([0., 0.2, 0.4, 0.6, 0.8, 1.]);
plt.tight_layout();